In [1]:
import os

dir='/root/autodl-tmp/cache/'
os.environ["HF_DATASETS_CACHE"] = dir
os.environ["HF_HOME"] = dir
os.environ["HUGGINGFACE_HUB_CACHE"] = dir
os.environ["TRANSFORMERS_CACHE"] = dir
os.environ["HF_ENDPOINT"]='https://hf-mirror.com'

import torch
from watermark.auto_watermark import AutoWatermark
from utils.transformers_config import TransformersConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
# Device
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
transformers_config = TransformersConfig(
    model=AutoModelForCausalLM.from_pretrained('Qwen/Qwen2.5-0.5B').to(device),
    tokenizer=AutoTokenizer.from_pretrained('Qwen/Qwen2.5-0.5B'),
    # vocab_size=50272,
    device=device,
    max_new_tokens=100,
    min_length=20,
    do_sample=True,
    # no_repeat_ngram_size=4
)


In [4]:
wm1 = AutoWatermark.load(
    'KGW', 
    algorithm_config='config/KGW.json',
    transformers_config=transformers_config
)
# wm2 = AutoWatermark.load(
#     'KGW', 
#     algorithm_config='config/KGW_.json',
#     transformers_config=transformers_config
# )


In [ ]:
import paraphrasing
prompt1 ="Paris is a city with "
wm=paraphrasing.Watermarker(wm1)
wm1_ans,sc = wm.generate_wm_text(prompt1)
print(wm1_ans)
print(sc)

In [ ]:
paraphraser=paraphrasing.ParaphrasingAttacker(config='config/Qwen.toml')
# print(paraphraser.args)
ans=paraphraser.pipeline_paraphrase(wm1_ans)
print(ans)
para_sc=wm.detect_score(ans)
print(para_sc)

In [ ]:
paraphrasing.Watermarker.contact_viz(
    wm.viz(wm1_ans,savepath='viz/KGW_pre.png'),
    wm.viz(ans,savepath='viz/KGW_post.png')
)

In [8]:
import pandas as pd
d=dict(
    ori_text=[wm1_ans],
    ori_score=[sc],
    para_text=[ans],
    para_score=[para_sc],
    
)
df=pd.DataFrame(d)
df.to_csv('paraphrase_attack.csv')